# Examples

Dynamite has several submodules for various functionality. This page was generated from a Jupyter Notebook, and all of the TeX output is automically generated by dynamite when interacting through a notebook. See `docs/Examples.ipynb` in the source tree to view this file in notebook form, and run the examples yourself!

If you are looking for another solid example in the form of a script, see `benchmark/benchmarking.py`.

We'll start with the core functionality, building Hamiltonians.

### Hamiltonian Building
We begin by importing the Pauli sigma matrices:

In [1]:
from dynamite.operators import Sigmax, Sigmay, Sigmaz

Hamiltonians in `dynamite` are represented as sums and products of Pauli sigma matrices on individual spin sites, with implied tensor product of identity operators on all other sites. For example a single Sigmax operator on site 1 (indexing starts at 0):

In [2]:
Sigmax(1)

represents the operator $I_0 \otimes \sigma^x_1 \otimes I_2 \otimes \ldots \otimes I_L$. Note that we haven't set the length of the spin chain yet---we don't need to! The identities are implied out to whatever arbitrary length $L$.

We can add, subtract, multiply, do whatever with Pauli operators on various sites, and in this way construct any arbitrary Hermitian matrix:

In [3]:
a = Sigmax(0) * Sigmay(1) - 0.2*Sigmaz(0)
a

We can also use the Sum and Product classes to perform a sum of a list of terms explicitly, like so:

In [4]:
from dynamite.operators import Sum, Product
Sum([Sigmax(0),Sigmax(1)])

A nice Pythonic thing to do is to use a generator as the argument to Sum() or Product(), for example to build the **dipolar interaction**:

In [5]:
Sum(s(0)*s(1) for s in [Sigmax,Sigmay,Sigmaz])

Often, the same operator should be translated along each position in the spin chain, and `dynamite` makes that easy with the classes `IndexSum` and `IndexProduct`. For example, to take the dipolar interaction above and apply it to every spin and its neighbor is simply:

In [6]:
from dynamite.operators import IndexSum, IndexProduct

In [7]:
# dipolar interaction between spins 0 and 1
dipolar = Sum(s(0)*s(1) for s in [Sigmax,Sigmay,Sigmaz])

# translate that along the spin chain
IndexSum(dipolar)

Note that the spin chain length L is *still* symbolic! We can set it explicitly if we want, in several ways:

In [8]:
H = IndexSum(dipolar,L=20) # set in constructor

# or after building it (both do the same thing)
H.L = 20
H.set_length(20)

H

The class IndexProduct has similar functionality but takes the product of the transposed operators. For example, a matrix that will flip all spins can be implemented as:

In [9]:
IndexProduct(Sigmax())

One should note that the interaction doesn't need to be nearest-neighbor, but it should include something on spin 0:

In [10]:
IndexSum(Sigmaz(0)*Sigmaz(2)) # next-nearest-neighbor interaction

Applying the interaction to only some portion of the spin chain is accomplished by the `min_i` and `max_i` arguments:

In [11]:
IndexSum(Sigmaz(0),min_i=4,max_i=8)

### Evolution

#### Building an initial state

We must give dynamite some initial state to evolve under our Hamiltonian. Product states can be generated with the `dynamite.tools.build_state` function, which takes as arguments the spin chain length and the spin configuration for the product state. For example:

In [12]:
from dynamite.tools import build_state

# specify state with a string of spins up and down
s = build_state(L=10,state='UDDUDUDUUD')

In [13]:
# specify state as an integer (whose binary
# representation is the spin configuration)
s = build_state(L=18,state=341)
print(bin(341))

0b101010101


#### Evolving a state

Then, the state can be evolved very easily under some Hamiltonian, say our dipolar interaction from earlier:

In [14]:
H = IndexSum(dipolar)
H.L = 18

# evolve for t=1.0
result = H.evolve(s,t=1.0)

Now we have a result vector, and can do whatever we want with it, like find $\left<S^z_0\right>$:

In [15]:
sz0 = Sigmaz(0,L=18)
result.dot(sz0*result)

(-0.12670366969556432+0j)

### Eigensolving

Finding eigenvalues and eigenvectors is also really easy:

In [16]:
H.eigsolve()

array([-31.18804427+0.j])

By default, `dynamite` solves for the ground state eigenvalue. SLEPc, upon which `dynamite` is based, is powerful because it can find just a small subset of eigenpairs very quickly. It is not the appropriate tool to find the whole spectrum!

It is easy to get eigenvectors as well, and one can specify how many eigenpairs are desired:

In [17]:
H.eigsolve(nev=4,getvecs=True)

(array([-31.18804427+0.j, -30.39713954+0.j, -29.41917996+0.j,
        -29.15883835+0.j]),
  <petsc4py.PETSc.Vec at 0x7fd56282d200>])